# Object Detection

In [ ]:
import torch
import numpy as np
import cv2
from time import time
import sys


class ObjectDetection:
    """
    The class performs generic object detection on a video file.
    It uses yolo5 pretrained model to make inferences and opencv2 to manage frames.
    Included Features:
    1. Reading and writing of video file using  Opencv2
    2. Using pretrained model to make inferences on frames.
    3. Use the inferences to plot boxes on objects along with labels.
    Upcoming Features:
    """
    def __init__(self, input_file, out_file="Labeled_Video.avi"):
        """
        :param input_file: provide youtube url which will act as input for the model.
        :param out_file: name of a existing file, or a new file in which to write the output.
        :return: void
        """
        self.input_file = input_file
        self.model = self.load_model()
        self.model.conf = 0.4 # set inference threshold at 0.3
        self.model.iou = 0.3 # set inference IOU threshold at 0.3
        self.model.classes = [0] # set model to only detect "Person" class
        self.out_file = out_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def get_video_from_file(self):
        """
        Function creates a streaming object to read the video from the file frame by frame.
        :param self:  class object
        :return:  OpenCV object to stream video frame by frame.
        """
        cap = cv2.VideoCapture(self.input_file)
        assert cap is not None
        return cap

    def load_model(self):
        """
        Function loads the yolo5 model from PyTorch Hub.
        """
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model

    def score_frame(self, frame):
        """
        function scores each frame of the video and returns results.
        :param frame: frame to be infered.
        :return: labels and coordinates of objects found.
        """
        self.model.to(self.device)
        results = self.model([frame])
        labels, cord = results.xyxyn[0][:, -1].to('cpu').numpy(), results.xyxyn[0][:, :-1].to('cpu').numpy()
        return labels, cord

    def plot_boxes(self, results, frame):
        """
        plots boxes and labels on frame.
        :param results: inferences made by model
        :param frame: frame on which to  make the plots
        :return: new frame with boxes and labels plotted.
        """
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
            bgr = (0, 0, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 3)
            label = f"{int(row[4]*100)}"
            cv2.putText(frame, label, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)
            cv2.putText(frame, f"Total Targets: {n}", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        return frame

    def __call__(self):
        player = self.get_video_from_file() # create streaming service for application
        assert player.isOpened()
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 20, (x_shape, y_shape))
        fc = 0
        fps = 0
        tfc = int(player.get(cv2.CAP_PROP_FRAME_COUNT))
        tfcc = 0
        while True:
            fc += 1
            start_time = time()
            ret, frame = player.read()
            if not ret:
                break
            results = self.score_frame(frame)
            frame = self.plot_boxes(results, frame)
            end_time = time()
            fps += 1/np.round(end_time - start_time, 3)
            if fc == 10:
                fps = int(fps / 10)
                tfcc += fc
                fc = 0
                per_com = int(tfcc / tfc * 100)
                print(f"Frames Per Second : {fps} || Percentage Parsed : {per_com}")
            out.write(frame)
        player.release()

In [ ]:
link = sys.argv[1]
output_file = sys.argv[2]
a = ObjectDetection("C:/Users/sunfo/Downloads/이상행동 CCTV 영상/02.싸움(fight)/insidedoor_03/37-4/37-4_cam01_fight01_place02_night_spring.mp4", 
                   'C:/Users/sunfo/OneDrive/바탕 화면/detected_37-4_cam01_fight01_place02_night_spring.mp4')
a()

# Custom Training
### test_size=0.5 / epochs 20

violence,NonViolence detection 
https://universe.roboflow.com/nuscrimesocietydatasets/violence-9gmjx

In [1]:
from glob import glob

img_list = glob('C:/sooeun/DATAexam/ConvergenceProject-CareSystem/roboflow/combined/images/*.jpg')
print(len(img_list))

2999


In [2]:
# ! pip install numpy --upgrade

In [3]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.5, random_state=2000)
print(len(train_img_list), len(val_img_list))

C:\Users\ppi06\anaconda3\envs\data_venv\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


1499 1500


In [4]:
# train_img_list = glob("C:/sooeun/DATAexam/ConvergenceProject-CareSystem/roboflow/train/images/*.jpg")
# val_img_list = glob("C:/sooeun/DATAexam/ConvergenceProject-CareSystem/roboflow/valid/images/*.jpg")

In [5]:
# len(train_img_list)

In [6]:
# train, val 정보에 대한 txt 파일 생성
with open('C:/sooeun/DATAexam/ConvergenceProject-CareSystem/roboflow/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')
    
with open('C:/sooeun/DATAexam/ConvergenceProject-CareSystem/roboflow/val.txt', 'w') as f:
    f.write('\n'.join(val_img_list) + '\n')

In [7]:
# yaml 파일 수정
import yaml

with open("C:/sooeun/DATAexam/ConvergenceProject-CareSystem/roboflow/data.yaml", 'r') as f:
    data = yaml.safe_load(f)
    
print(data)

data['train'] = "C:/sooeun/DATAexam/ConvergenceProject-CareSystem/roboflow/train.txt"
data['val'] = "C:/sooeun/DATAexam/ConvergenceProject-CareSystem/roboflow/val.txt"

with open("C:/sooeun/DATAexam/ConvergenceProject-CareSystem/roboflow/data.yaml", 'w') as f:
    yaml.dump(data, f)

print(data)

{'names': ['NonViolence', 'Violence'], 'nc': 2, 'train': 'C:/sooeun/DATAexam/ConvergenceProject-CareSystem/roboflow/train.txt', 'val': 'C:/sooeun/DATAexam/ConvergenceProject-CareSystem/roboflow/val.txt'}
{'names': ['NonViolence', 'Violence'], 'nc': 2, 'train': 'C:/sooeun/DATAexam/ConvergenceProject-CareSystem/roboflow/train.txt', 'val': 'C:/sooeun/DATAexam/ConvergenceProject-CareSystem/roboflow/val.txt'}


## ---여기서부터 기동---

In [8]:
from glob import glob

In [9]:
# yolo 경로로 이동
# yolov5s로 training
%cd C:\Users\ppi06\yolov5
!python train.py --img 416 --batch 16 --epochs 20 --data C:\sooeun\DATAexam\ConvergenceProject-CareSystem\roboflow\data.yaml --cfg C:/Users/ppi06/yolov5/models/yolov5s.yaml --weights yolov5s.pt --name 230116_roboflow_dataset_test1

C:\Users\ppi06\yolov5
^C


### best.pt 경로 확인 후 변경 

In [ ]:
# 학습된 모델로 detecting
!python detect.py --source C:/sooeun/DATAexam/ConvergenceProject-CareSystem/12-6_cam01_assault01_place09_day_summer.mp4 --weights C:/Users/ppi06/yolov5/runs/train/230116_roboflow_dataset_test12/weights/best.pt